In [1]:
#Imports---------------------------------------------------------------
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
#%matplotlib inline
#%config InlineBackend.figure_format = 'retina'

import glob
import numpy as np
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
#Main pipeline for multiclass classification
#import libraries
import numpy as np
from time import time
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from tqdm import tqdm
from sklearn.model_selection import KFold
from torchvision import models, transforms
import pickle
import os

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
import torch.nn.functional as F

K=10
arc='wideresnet'
#loss_func = nn.CrossEntropyLoss()
#loss_func =F.cross_entropy()

lr=0.03
b_s=64
ep=50
PATH='/content/drive/My Drive/ML/model.pth'
PATH2='/content/drive/My Drive/ML/files/unl.pth'

#APAGAR-------------------------------------
#with open('/content/drive/My Drive/ML/Resultados/readme.txt', 'w') as f:
#    f.write('')
#with open('/content/drive/My Drive/ML/Resultados/readme.txt', 'a') as f:
#    f.write('------------------------------------------------Self-Supervised\nParametros:\n\nEpochs'+str(ep)+'\nArc:'+arc+'\nLearning rate:'+str(lr)+'\nBatch size:'+str(b_s))

In [4]:
##Datasets como vão buscar no Fix-match Pytorch-------------------------------------------------------------------------------
##----------------------------------------------------------------------------------------------------------------------------

import logging
import math

import numpy as np
from PIL import Image
from torchvision import datasets
from torchvision import transforms

#from .randaugment import RandAugmentMC

logger = logging.getLogger(__name__)

cifar10_mean = (0.4914, 0.4822, 0.4465)
cifar10_std = (0.2471, 0.2435, 0.2616)
cifar100_mean = (0.5071, 0.4867, 0.4408)
cifar100_std = (0.2675, 0.2565, 0.2761)
normal_mean = (0.5, 0.5, 0.5)
normal_std = (0.5, 0.5, 0.5)


def get_cifar10(num_labeled,num_classes, root):
    transform_labeled = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(size=32,
                              padding=int(32*0.125),
                              padding_mode='reflect'),
        transforms.ToTensor(),
        transforms.Normalize(mean=cifar10_mean, std=cifar10_std)
    ])
    transform_val = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=cifar10_mean, std=cifar10_std)
    ])
    base_dataset = datasets.CIFAR10(root, train=True, download=True)

    train_labeled_idxs, train_unlabeled_idxs = x_u_split(
        num_labeled,num_classes, base_dataset.targets)

    train_labeled_dataset = CIFAR10SSL(
        root, train_labeled_idxs, train=True,
        transform=transform_labeled)

    train_unlabeled_dataset = CIFAR10SSL(
        root, train_unlabeled_idxs, train=True,
        transform=TransformFixMatch(mean=cifar10_mean, std=cifar10_std))

    test_dataset = datasets.CIFAR10(
        root, train=False, transform=transform_val, download=False)

    return train_labeled_dataset, train_unlabeled_dataset, test_dataset


def get_cifar100(args, root):

    transform_labeled = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(size=32,
                              padding=int(32*0.125),
                              padding_mode='reflect'),
        transforms.ToTensor(),
        transforms.Normalize(mean=cifar100_mean, std=cifar100_std)])

    transform_val = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=cifar100_mean, std=cifar100_std)])

    base_dataset = datasets.CIFAR100(
        root, train=True, download=True)

    train_labeled_idxs, train_unlabeled_idxs = x_u_split(
        args, base_dataset.targets)

    train_labeled_dataset = CIFAR100SSL(
        root, train_labeled_idxs, train=True,
        transform=transform_labeled)

    train_unlabeled_dataset = CIFAR100SSL(
        root, train_unlabeled_idxs, train=True,
        transform=TransformFixMatch(mean=cifar100_mean, std=cifar100_std))

    test_dataset = datasets.CIFAR100(
        root, train=False, transform=transform_val, download=False)

    return train_labeled_dataset, train_unlabeled_dataset, test_dataset


def x_u_split(num_labeled,num_classes, labels):
    label_per_class = num_labeled // num_classes
    labels = np.array(labels)
    labeled_idx = []
    # unlabeled data: all data (https://github.com/kekmodel/FixMatch-pytorch/issues/10)
    unlabeled_idx = np.array(range(len(labels)))
    for i in range(num_classes):
        idx = np.where(labels == i)[0]
        idx = np.random.choice(idx, label_per_class, False)
        labeled_idx.extend(idx)
    labeled_idx = np.array(labeled_idx)
    assert len(labeled_idx) == num_labeled

   
    np.random.shuffle(labeled_idx)
    return labeled_idx, unlabeled_idx



class TransformFixMatch(object):
    def __init__(self, mean, std):
        self.weak = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(size=32,
                                  padding=int(32*0.125),
                                  padding_mode='reflect')])
        self.strong = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(size=32,
                                  padding=int(32*0.125),
                                  padding_mode='reflect'),
            torchvision.transforms.RandAugment(2,10)])
        self.normalize = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)])

    def __call__(self, x):
        weak = self.weak(x)
        strong = self.strong(x)
        return self.normalize(weak), self.normalize(strong)


class CIFAR10SSL(datasets.CIFAR10):
    def __init__(self, root, indexs, train=True,
                 transform=None, target_transform=None,
                 download=False):
        super().__init__(root, train=train,
                         transform=transform,
                         target_transform=target_transform,
                         download=download)
        if indexs is not None:
            self.data = self.data[indexs]
            self.targets = np.array(self.targets)[indexs]

    def __getitem__(self, index):
        img, target = self.data[index], self.targets[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target


class CIFAR100SSL(datasets.CIFAR100):
    def __init__(self, root, indexs, train=True,
                 transform=None, target_transform=None,
                 download=False):
        super().__init__(root, train=train,
                         transform=transform,
                         target_transform=target_transform,
                         download=download)
        if indexs is not None:
            self.data = self.data[indexs]
            self.targets = np.array(self.targets)[indexs]

    def __getitem__(self, index):
        img, target = self.data[index], self.targets[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target


DATASET_GETTERS = {'cifar10': get_cifar10,
                   'cifar100': get_cifar100}

In [9]:
##Cria datasets-------------------------------------------------------------------------------------------------------------  
num_labeled=250
num_classes=10

labeled_dataset, unlabeled_dataset, test_dataset = get_cifar10(num_labeled,num_classes,'./data')

labeled_dataset, unlabeled_dataset, test_dataset = DATASET_GETTERS['cifar10'](num_labeled,num_classes, './data')
print(len(labeled_dataset))
print(len(unlabeled_dataset))
print(len(test_dataset))


Files already downloaded and verified
Files already downloaded and verified
250
50000
10000


In [15]:
## Cria Dataloaders----------------------------------------------------------------------------------------------

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_sampler = RandomSampler #if args.local_rank == -1 else DistributedSampler
labeled_trainloader = DataLoader(
        labeled_dataset,
        sampler=train_sampler(labeled_dataset),
        batch_size=32,
        #num_workers=4,
        drop_last=True)

unlabeled_trainloader = DataLoader(
        unlabeled_dataset,
        sampler=train_sampler(unlabeled_dataset),
        batch_size=32*7,
        #num_workers=4,
        drop_last=True)

test_loader = DataLoader(
        test_dataset,
        sampler=SequentialSampler(test_dataset),
        batch_size=32)
        #num_workers=4)


print(len(labeled_trainloader))
print(len(unlabeled_trainloader))

7
223


In [ ]:
##Dataset CIFAR 10 Feito por mim-------------------------------------------------------------------------------
##----------------------------------------------------------------------------------------------------------------------------

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform_strong = transforms.Compose([
 #transforms.Resize(224),                    #[2]
 #transforms.CenterCrop(224),                #[3]
 transforms.RandomHorizontalFlip(),
 transforms.RandomVerticalFlip(),
 torchvision.transforms.RandAugment(2,5),
 transforms.ToTensor(),                     #[4]
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
counter=[0]*10
labset=[]
unlabset=[]
for i in range (0,50000):
  for z in range(0,10):
    if(trainset[i][1]==z):
      if(counter[z]<25):
        counter[z]=counter[z]+1
        labset.append(trainset[i])
      else:
        unlabset.append(trainset[i])  
print(counter)
print(len(labset))
print(len(unlabset))  
labloader = torch.utils.data.DataLoader(labset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_strong)
counter=[0]*10
labset2=[]
unlabset=[]
for i in range (0,50000):
  for z in range(0,10):
    if(trainset[i][1]==z):
      if(counter[z]<25):
        counter[z]=counter[z]+1
        labset2.append(trainset[i])
      else:
        unlabset.append(trainset[i])  
print(counter)
print(len(labset))
print(len(unlabset))  
unlabloader = torch.utils.data.DataLoader(unlabset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


In [ ]:
##Dataseat Transform Unlabeled weak--------------------------------------------
transform_simple = transforms.Compose([            #[1]
 #transforms.Resize(224),                    #[2]
 #transforms.CenterCrop(224),                #[3]
 transforms.RandomHorizontalFlip(),
 transforms.RandomVerticalFlip(),
 transforms.ToTensor(),                     #[4]
 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_simple)
counter=[0]*10
labset3=[]
unlabset_w=[]
for i in range (0,50000):
  for z in range(0,10):
    if(trainset[i][1]==z):
      if(counter[z]<25):
        counter[z]=counter[z]+1
        labset.append(trainset[i])
      else:
        unlabset_w.append(trainset[i])  

labloader_w = torch.utils.data.DataLoader(unlabset_w, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [ ]:
##Versão com mu=7----------------------------------------------------------
labloader_w = torch.utils.data.DataLoader(unlabset_w, batch_size=batch_size*7,
                                          shuffle=True, num_workers=2)
unlabloader = torch.utils.data.DataLoader(unlabset, batch_size=batch_size*7,
                                          shuffle=True, num_workers=2)

In [11]:
## AlexNet usada no modelo normal---------------------------------------------------
class AlexNet(nn.Module):
    def __init__(self, num_classes=K):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(256 * 2 * 2, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, K),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 2 * 2)
        x = self.classifier(x)
        return x

In [12]:
##Wideresnet usada por eles-------------------------------------------------------------------------------
##----------------------------------------------------------------------------------------------------------------------------
import logging

import torch
import torch.nn as nn
import torch.nn.functional as F

logger = logging.getLogger(__name__)


def mish(x):
    """Mish: A Self Regularized Non-Monotonic Neural Activation Function (https://arxiv.org/abs/1908.08681)"""
    return x * torch.tanh(F.softplus(x))


class PSBatchNorm2d(nn.BatchNorm2d):
    """How Does BN Increase Collapsed Neural Network Filters? (https://arxiv.org/abs/2001.11216)"""

    def __init__(self, num_features, alpha=0.1, eps=1e-05, momentum=0.001, affine=True, track_running_stats=True):
        super().__init__(num_features, eps, momentum, affine, track_running_stats)
        self.alpha = alpha

    def forward(self, x):
        return super().forward(x) + self.alpha


class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride, drop_rate=0.0, activate_before_residual=False):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes, momentum=0.001)
        self.relu1 = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes, momentum=0.001)
        self.relu2 = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.drop_rate = drop_rate
        self.equalInOut = (in_planes == out_planes)
        self.convShortcut = (not self.equalInOut) and nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride,
                                                                padding=0, bias=False) or None
        self.activate_before_residual = activate_before_residual

    def forward(self, x):
        if not self.equalInOut and self.activate_before_residual == True:
            x = self.relu1(self.bn1(x))
        else:
            out = self.relu1(self.bn1(x))
        out = self.relu2(self.bn2(self.conv1(out if self.equalInOut else x)))
        if self.drop_rate > 0:
            out = F.dropout(out, p=self.drop_rate, training=self.training)
        out = self.conv2(out)
        return torch.add(x if self.equalInOut else self.convShortcut(x), out)


class NetworkBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, out_planes, block, stride, drop_rate=0.0, activate_before_residual=False):
        super(NetworkBlock, self).__init__()
        self.layer = self._make_layer(
            block, in_planes, out_planes, nb_layers, stride, drop_rate, activate_before_residual)

    def _make_layer(self, block, in_planes, out_planes, nb_layers, stride, drop_rate, activate_before_residual):
        layers = []
        for i in range(int(nb_layers)):
            layers.append(block(i == 0 and in_planes or out_planes, out_planes,
                                i == 0 and stride or 1, drop_rate, activate_before_residual))
        return nn.Sequential(*layers)

    def forward(self, x):
        return self.layer(x)


class WideResNet(nn.Module):
    def __init__(self, num_classes, depth=28, widen_factor=2, drop_rate=0.0):
        super(WideResNet, self).__init__()
        channels = [16, 16*widen_factor, 32*widen_factor, 64*widen_factor]
        assert((depth - 4) % 6 == 0)
        n = (depth - 4) / 6
        block = BasicBlock
        # 1st conv before any network block
        self.conv1 = nn.Conv2d(3, channels[0], kernel_size=3, stride=1,
                               padding=1, bias=False)
        # 1st block
        self.block1 = NetworkBlock(
            n, channels[0], channels[1], block, 1, drop_rate, activate_before_residual=True)
        # 2nd block
        self.block2 = NetworkBlock(
            n, channels[1], channels[2], block, 2, drop_rate)
        # 3rd block
        self.block3 = NetworkBlock(
            n, channels[2], channels[3], block, 2, drop_rate)
        # global average pooling and classifier
        self.bn1 = nn.BatchNorm2d(channels[3], momentum=0.001)
        self.relu = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.fc = nn.Linear(channels[3], num_classes)
        self.channels = channels[3]

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight,
                                        mode='fan_out',
                                        nonlinearity='leaky_relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1.0)
                nn.init.constant_(m.bias, 0.0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 0.0)

    def forward(self, x):
        out = self.conv1(x)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.relu(self.bn1(out))
        out = F.adaptive_avg_pool2d(out, 1)
        out = out.view(-1, self.channels)
        return self.fc(out)


In [18]:
#Give to the model the architecture and number of classes (K)
model = WideResNet(K)
#put the model on 'GPU' or 'CPU'
model = model.to(device)
#Define optimizer and scheduler 
optimizer = optim.Adam(model.parameters(),lr )

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True)

In [ ]:
import torch.nn.functional as F

def interleave(x, size):
    s = list(x.shape)
    return x.reshape([-1, size] + s[1:]).transpose(0, 1).reshape([-1] + s[1:])


def de_interleave(x, size):
    s = list(x.shape)
    return x.reshape([size, -1] + s[1:]).transpose(0, 1).reshape([-1] + s[1:])

gama=0
thresh=0.95
running_loss=1010.0
train_acc=0.0
final_t_a=0.0
l_train=547*32
l_test=182*32
avg_acc=0.0
best_loss=1000.0
best_acc=0.0
torch.cuda.empty_cache()
best_acc_t=0
best_loss_t=0
PATH2='/content/drive/My Drive/ML/files/unl.pth'




for epoch in range(100):
    print('\n')
    print('Iteração:',epoch)
    print('loss:',running_loss)
    print('acc:',avg_acc)
  #  torch.save({
  #           'epoch': epoch,
  #           'model_state_dict': model.state_dict(),
  #          'optimizer_state_dict': optimizer.state_dict(),
  #          'loss': running_loss
  #          
  #          }, PATH2)
    end=0
    count=1
    #unlabeledw_iter = iter(labloader_w)
    unlabeled_iter = iter(unlabeled_trainloader)
    labeled_iter = iter(labeled_trainloader)

    if(epoch<=15):     
      labeled_iter = iter(labeled_trainloader)
    avg_acc=0.0
    running_loss=0.0
    waiting=0
    while (end==0):
     if(epoch>15): 
      count=count+1
      
   
      try:
       inputs, labels = labeled_iter.next()
      except StopIteration:
         end=1
         
      try:  
        (weak,strong),_ = unlabeled_iter.next()
        #weak,_ = unlabeledw_iter.next()
        
        
      except StopIteration:
        end=1  
        
      if(end==0):  
        inputs,labels = inputs.to(device),labels.to(device)
        weak,strong = weak.to(device),strong.to(device)
        optimizer.zero_grad()
        batch_size = inputs.shape[0]
        #labeled data---------------------------------------------
        inp = interleave(
                torch.cat((inputs, weak, strong)), 2*7+1).to(device)
        print(len(inp))
        logits = model(inp)
        logits = de_interleave(logits, 2*7+1)
        logits_x = logits[:batch_size]
        logits_u_w, logits_u_s = logits[batch_size:].chunk(2) 
        del logits
        
        #outputs=model(inputs)
        Lx = F.cross_entropy(logits_x, labels, reduction='mean')    
        #loss_l = loss_func(outputs, labels)
        _, predicted = torch.max(logits_x, 1)
        real=labels
        correct = (predicted == real).float().sum()
        avg_acc +=correct*100/(32*len(unlabeled_iter))


           
        #Unlabeled data----------------------------------------------------
        #outputs = model(weak)
        #outputs_s = model(strong)
        pseudo_label =  torch.softmax(logits_u_s,dim=-1)
        pseudo_label,index = torch.max(pseudo_label,dim=-1)
        mask = pseudo_label.ge(thresh).float()
        Lu = (F.cross_entropy(logits_u_s, index,reduction='none') * mask).mean()
        #loss_u = loss_func(outputs, index)*mask
     
        #Calculate both losses------------------------------------------------
        loss = Lx + gama*Lu
        #loss = loss_l + gama*loss_u
        loss.backward()
        optimizer.step()     
        
        running_loss += loss.item()
        print('.',end='')

        if avg_acc>=best_acc and running_loss<=best_loss :
             best_acc=avg_acc
             best_loss=running_loss
     else:
        count=count+1
        if(count==112):
          end=1
     
        try:
          inputs, labels = labeled_iter.next()
          #weak,_ = unlabeledw_iter.next()
          #strong,_ = unlabeleds_iter.next()
          optimizer.zero_grad()

        except StopIteration:
          end=1
        if(end==0):  
          inputs,labels = inputs.to(device),labels.to(device)
          #weak,strong = weak.to(device),strong.to(device)

          #labeled data---------------------------------------------
          outputs = model(inputs)  
          Lx = F.cross_entropy(outputs, labels, reduction='mean')    
          #loss_l = loss_func(outputs, labels)
          _, predicted = torch.max(outputs, 1)
          real=labels
          correct = (predicted == real).float().sum()
          avg_acc +=correct*100/(32*len(labeled_iter))

      
          loss = Lx #+ gama*Lu
          #loss = loss_l + gama*loss_u
          loss.backward()
          optimizer.step()     
          
          running_loss += loss.item()
          print('.',end='')

          if avg_acc>=best_acc and running_loss<=best_loss :
             best_acc=avg_acc
             best_loss=running_loss    
    if(epoch>20):
      scheduler.step(running_loss)             

In [ ]:
running_loss=0.0
train_acc=0.0
final_t_a=0.0
l= len(test_loader)*32
avg_acc=0.0


##Test the dataset
model.eval()
for inputs,labels in test_loader:
          inputs,labels = inputs.to(device),labels.to(device)
          #weak,strong = weak.to(device),strong.to(device)

          #labeled data---------------------------------------------
          outputs = model(inputs)  
          Lx = F.cross_entropy(outputs, labels, reduction='mean')    
          #loss_l = loss_func(outputs, labels)
          _, predicted = torch.max(outputs, 1)
          real=labels
          correct = (predicted == real).float().sum()
          avg_acc +=correct*100/l
      
          loss = Lx #+ gama*Lu
          #loss = loss_l + gama*loss_u
          #loss.backward()
          #optimizer.step()     
          running_loss += loss.item() 
          print('.',end='')

print('\n')
print('loss:',running_loss)
print('acc:',avg_acc)
print('Finished Validation')

In [ ]:
for inputs,labels in labloader:
  print(labels)